In [1]:
import pandas as pd

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


# 검증 세트 

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()

In [5]:
target = wine['class'].to_numpy()

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size =0.2, random_state= 42)

In [8]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [9]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


## 모델을 만들고 평가하기

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
dt = DecisionTreeClassifier(random_state=42)

In [12]:
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [13]:
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증

In [14]:
from sklearn.model_selection import cross_validate

In [15]:
scores = cross_validate(dt, train_input, train_target)

In [16]:
print(scores)

{'fit_time': array([0.01599717, 0.01515579, 0.01712155, 0.02100492, 0.02066851]), 'score_time': array([0.00243068, 0.00100517, 0.00200033, 0.00300527, 0.00300336]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


## 검증 폴드의 점수

In [17]:
import numpy as np

In [18]:
np.mean(scores['test_score'])

0.855300214703487

In [19]:
from sklearn.model_selection import StratifiedKFold

In [20]:
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

## 훈련 세트를 섞은 후 10-폴드 교차 검증 수행

In [21]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8574181117533719

# 하이퍼파라미터 튜닝

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs =-1)

In [26]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [27]:
dt = gs.best_estimator_

In [28]:
dt.score(train_input, train_target)

0.9615162593804117

In [29]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [31]:
#각 매개변수에서 수행한 교차 검증의 평균 점수
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [34]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

## 조금 더 복잡한 매개변수 조합 탐색

In [35]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
            'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
         }

In [37]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [40]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [41]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [42]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

# 랜덤 서치

In [43]:
from scipy.stats import uniform, randint

In [51]:
rgen = randint(0,10)
rgen.rvs(10)

array([5, 7, 0, 0, 4, 4, 8, 0, 9, 0], dtype=int64)

In [54]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([100, 119,  80, 121,  76, 107,  85, 111,  98, 103], dtype=int64))

In [55]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.75222803, 0.9138605 , 0.57967532, 0.27801327, 0.47826254,
       0.27667467, 0.95716156, 0.2408838 , 0.45240214, 0.65157944])